# Compare runtime

In [1]:
#include("../netx.jl")
push!(LOAD_PATH, "../"); using netx
#import .netx

using BenchmarkTools

┌ Info: Precompiling netx [top-level]
└ @ Base loading.jl:1278


## Path to data

In [2]:
path_to = "../ER/"
folders = String["TYPE1", "TYPE2", "TYPE4", "TYPE8"]

sizes = Int[512, 1024, 2048, 4096, 8192, 16384]
k_mean = Float64[1.0, 2.0, 4.0, 8.0]
ntype = Int[1, 2, 4, 8]

4-element Array{Int64,1}:
 1
 2
 4
 8

In [3]:
function process_network(N::Int, k::Float64, ne_type::Int, samp::Int)
    fname = "$(path_to)TYPE$(ntype[ne_type])/ER_N$(N)_k$(k)_samp$(samp).txt"
    edges, eprops = netx.process_edgefile(fname, true)
    fmt_eprops = netx.process_eprops(eprops, ["regulation"])
    edgetype_prop = parse.(Int, fmt_eprops["regulation"])
    
    # -- Create network and set the edgetypes as an edge property --
    g = netx.graph_from_edgelist(edges, true)
    netx.set_edges_properties("edgetype", edgetype_prop, g)
    return g
end

function process_samples(N::Int, ne_type::Int)
    nsamp = 30
    runtime = [ Float64[0.0 for j in 1:nsamp] for i in 1:4 ]
    mem = [ Float64[0.0 for j in 1:nsamp] for i in 1:4 ]
    for (j, k_mean) in enumerate(Float64[1.0, 2.0, 4.0, 8.0])
        print("mean degree = $(k_mean)\n")
        for i in 1:nsamp
            print("$i ")
            g = process_network(N, k_mean, ne_type, i)
            b = @benchmarkable netx.fast_fibration($g)
            tune!(b)
            res = minimum(run(b))
            runtime[j][i] = res.time*1e-9
            mem[j][i] = res.memory*1e-6
        end
        print("\n")
    end
    return runtime, mem
end

process_samples (generic function with 1 method)

In [66]:
N512_T1_time, N512_T1_memory = process_samples(512, 1)
N1024_T1_time, N1024_T1_memory = process_samples(1024, 1)
N2048_T1_time, N2048_T1_memory = process_samples(2048, 1)
N4096_T1_time, N4096_T1_memory = process_samples(4096, 1)
N8192_T1_time, N8192_T1_memory = process_samples(8192, 1)
N16384_T1_time, N16384_T1_memory = process_samples(16384, 1)
N32768_T1_time, N32768_T1_memory = process_samples(32768, 1)
return

mean degree = 1.0
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 
mean degree = 2.0
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 
mean degree = 4.0
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 
mean degree = 8.0
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 
mean degree = 1.0
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 
mean degree = 2.0
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 
mean degree = 4.0
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 
mean degree = 8.0
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 
mean degree = 1.0
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 
mean degree = 2.0
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 


LoadError: StackOverflowError:

In [71]:
function save_perfomance(name::String, sizes::Array{Int}, 
                         arr_time::Array{Array{Float64}}, arr_space::Array{Array{Float64}})
    open("$(name).txt", "w") do f
        write(f, "N\tk1_time\tk1_space\tk2_time\tk2_space\tk4_time\tk4_space\tk8_time\tk8_space\n")
        for (i, N) in sizes
            write(f, "$(N)")
            for j in 1:4
                write(f, "\t$(arr_time[i][j])\t$(arr_space[i][j])")
            end
            write(f,"\n")
        end
    end
end

function gen_mean(arr_time::Array{Array{Float64,1},1}, arr_space::Array{Array{Float64,1},1})
    new_time = [ 0.0 for j in 1:4 ]
    new_space = [ 0.0 for j in 1:4 ]
    for j in 1:4
        new_time[j] = mean(arr_time[j])
        new_space[j] = mean(arr_space[j])
    end
    return new_time, new_space
end

gen_mean (generic function with 2 methods)

In [74]:
new512_time, new512_space = gen_mean(N512_T1_time, N512_T1_memory)
new1024_time, new1024_space = gen_mean(N1024_T1_time, N1024_T1_memory)
new2048_time, new2048_space = gen_mean(N2048_T1_time, N2048_T1_memory)
new4096_time, new4096_space = gen_mean(N4096_T1_time, N4096_T1_memory)
new8192_time, new8192_space = gen_mean(N8192_T1_time, N8192_T1_memory)
#new16384_time, new16384_space = gen_mean(N16384_T1_time, N16384_T1_memory)
#new32768_time, new32768_space = gen_mean(N32768_T1_time, N32768_T1_memory)

sizes = Int[512, 1024, 2048, 4096, 8192, 16384, 32768]
arr_time = [new512_time,new1024_time,new2048_time,new4096_time,
            new8192_time]
arr_space = [new512_space,new1024_space,new2048_space,new4096_space,
            new8192_space]

5-element Array{Array{Float64,1},1}:
 [12.108447199999999, 19.4991664, 24.403565866666664, 28.112928266666664]
 [40.381243733333335, 68.79199093333334, 71.20931146666668, 73.70553866666665]
 [148.04283706666672, 263.7217592, 285.0775285333333, 190.4514957333333]
 [566.4277525333333, 1032.6660141333332, 1109.7759226666665, 555.1544013333332]
 [2231.9821703999996, 2974.8444264000004, 4371.447904533333, 2561.7676824000005]

In [76]:
arr_time

5-element Array{Array{Float64,1},1}:
 [0.008073264900000003, 0.0138916369, 0.016788516099999998, 0.015836507133333337]
 [0.029403402333333332, 0.06068266516666667, 0.0593510385, 0.05305222343333335]
 [0.12623453773333332, 0.23955023153333332, 0.27620211550000007, 0.1629542039]
 [0.5555732392666667, 0.9123979131000001, 1.1306037867000003, 0.5888000273333334]
 [2.5390206883333337, 2.957376237033334, 4.991906761300001, 2.7674409773999997]

In [41]:
out.gctimes

13-element Array{Float64,1}:
 6.4929507e7
 6.5049579e7
 7.7447115e7
 7.3743377e7
 7.8182339e7
 8.2424351e7
 7.983025e7
 9.0463789e7
 9.0254223e7
 9.9398462e7
 9.4692771e7
 8.5598788e7
 1.17096382e8

In [56]:
d = minimum(out)

BenchmarkTools.TrialEstimate: 
  time:             317.044 ms
  gctime:           64.930 ms (20.48%)
  memory:           316.64 MiB
  allocs:           5334104

In [4]:
g = process_network(16384, 8.0, 4, 1)
g1 = process_network(16384, 8.0, 4, 1)
netx.transpose_graph(g1)
return
#time = netx.dfs_search(g)

In [13]:
color, dist, parent, finished = netx.dfs_search(g)

([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1639, 12570, 399, 12689, 14279, 6165, 12047, 4007, 915  …  5010, 10798, 1199, 9801, 6914, 13721, 4893, 324, 7357, 4177], [-1, 12208, 994, 6441, 13228, 3462, 13737, 818, 12234, 16323  …  2492, 10283, 9323, 2403, 676, 1423, 9422, 2474, 11696, 11945], [32760, 31118, 19239, 32362, 19116, 14292, 26512, 19942, 28726, 31844  …  27705, 21419, 31558, 22564, 25725, 17976, 27824, 32437, 25250, 28552])

In [20]:
function find_root(node::Int, parent::Array{Int})
    par = parent[node]
    r = node
    while par!=-1
        r = par
        par = parent[par]
    end
    return r
end

roots = Int[]
for v in 1:length(g.vertices)
    push!(roots, find_root(v, parent))
end
roots = collect(Int, Set(roots))

5-element Array{Int64,1}:
  3808
 13643
  1276
     1
 16202

In [10]:
print(node1_t_tgt)

netx.Edge[netx.Edge(1, 3158, 1, true), netx.Edge(2, 6628, 1, true), netx.Edge(3, 7982, 1, true), netx.Edge(4, 11581, 1, true), netx.Edge(5, 13416, 1, true), netx.Edge(6, 15773, 1, true)]